# Ebemdding pipeline
Bioacoustics Model Zoo: https://github.com/kitzeslab/bioacoustics-model-zoo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bioacoustics_model_zoo as bmz

model = bmz.BirdNET()

2025-05-26 10:39:44.030818: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 10:39:48.206112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-26 10:39:48.977556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-26 10:39:49.286037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-26 10:39:50.642565: I tensorflow/core/platform/cpu_feature_guar

File BirdNET_GLOBAL_6K_V2.4_Labels_af.txt already exists; skipping download.


/storage/home/hcoda1/9/mgustineli3/clef/birdclef-2025/.venv/lib/python3.10/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

                    The function can also set the returned object's .constructor_name to the registered string key in ARCH_DICT
                    to avoid this warning and ensure it is reloaded correctly by opensoundscape.ml.load_model().

                    See opensoundscape.ml.cnn_architectures module for examples of constructor functions
                    
  warnings.wa

downloading model from URL...
File BirdNET_GLOBAL_6K_V2.4_Model_FP16.tflite already exists; skipping download.


In [3]:
bmz.list_models()

{'BirdNET': bioacoustics_model_zoo.birdnet.BirdNET,
 'YAMNet': bioacoustics_model_zoo.yamnet.YAMNet,
 'Perch': bioacoustics_model_zoo.perch.Perch,
 'HawkEars': bioacoustics_model_zoo.hawkears.hawkears.HawkEars,
 'BirdSetConvNeXT': bioacoustics_model_zoo.bmz_birdset.bmz_birdset_convnext.BirdSetConvNeXT,
 'BirdSetEfficientNetB1': bioacoustics_model_zoo.bmz_birdset.bmz_birdset_efficientnetB1.BirdSetEfficientNetB1,
 'RanaSierraeCNN': bioacoustics_model_zoo.rana_sierrae_cnn.RanaSierraeCNN}

In [4]:
models = {
    "BirdNET": {"window": 3.0, "embed_size": 1024},
    "YAMNet": {"window": 0.96, "embed_size": 1024},
    "Perch": {"window": 5.0, "embed_size": 1280},
    "HawkEars": {"window": 3.0, "embed_size": 2048},
    "BirdSetConvNeXT": {"window": 5.0, "embed_size": 1024},
    "BirdSetEfficientNetB1": {"window": 5.0, "embed_size": 1280},
    "RanaSierraeCNN": {"window": 2.0, "embed_size": 512},
}

In [23]:
import pandas as pd

# create pandas dataframe
model_df = pd.DataFrame.from_dict(models, orient="index")
model_df.reset_index(inplace=True)
model_df.columns = ["model", "window", "embed_size"]
model_df.head(10)

,model,window,embed_size
0,BirdNET,3.00,1024
1,YAMNet,0.96,1024
2,Perch,5.00,1280
3,HawkEars,3.00,2048
4,BirdSetConvNeXT,5.00,1024
5,BirdSetEfficientNetB1,5.00,1280
6,RanaSierraeCNN,2.00,512


### read raw birdcall data

In [43]:
from pathlib import Path

raw_root = Path("~/p-dsgt_clef2025-0/shared/birdclef/raw/birdclef-2025").expanduser()
! ls {raw_root}
soundscape_root = raw_root / "train_audio"
! ls {soundscape_root} | head
soundscapes = sorted(soundscape_root.glob("**/*.ogg"))
display(len(soundscapes))

recording_location.txt	taxonomy.csv	  train_audio  train_soundscapes
sample_submission.csv	test_soundscapes  train.csv
1139490
1192948
1194042
126247
1346504
134933
135045
1462711
1462737
1564122


28564

In [44]:
subset_soundscapes = soundscapes[:10]
len(subset_soundscapes)

10

### BirdNET model

In [45]:
# BirdNET inference
birdnet = bmz.BirdNET()

File BirdNET_GLOBAL_6K_V2.4_Labels_af.txt already exists; skipping download.


/storage/home/hcoda1/9/mgustineli3/clef/birdclef-2025/.venv/lib/python3.10/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

                    The function can also set the returned object's .constructor_name to the registered string key in ARCH_DICT
                    to avoid this warning and ensure it is reloaded correctly by opensoundscape.ml.load_model().

                    See opensoundscape.ml.cnn_architectures module for examples of constructor functions
                    
  warnings.wa

downloading model from URL...
File BirdNET_GLOBAL_6K_V2.4_Model_FP16.tflite already exists; skipping download.


In [46]:
# embed and predict using clip_step for every 1 second
embed, preds = birdnet.embed(subset_soundscapes[:3], return_preds=True, clip_step=1.0)
display(embed.head())
display(embed.shape)

  0%|          | 0/304 [00:00<?, ?it/s]

0     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       0.000000   
                                                   1.0        4.0       0.000000   
                                                   2.0        5.0       0.012751   
                                                   3.0        6.0       0.085640   
                                                   4.0        7.0       0.000000   

                                                                            1     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       0.174803   
                                                   1.0        4.0       0.106479   
                                                   2.0        5.0       0.317712   
                                                   3.0        6.0       0.482961   
                                                   4.0        7.0       0.258623   

                                                                            2     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       0.230387   
                                                   1.0        4.0       0.178000   
                                                   2.0        5.0       0.000000   
                                                   3.0        6.0       0.000000   
                                                   4.0        7.0       0.000000   

                                                                            3     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       0.175247   
                                                   1.0        4.0       0.270811   
                                                   2.0        5.0       0.604169   
                                                   3.0        6.0       0.635273   
                                                   4.0        7.0       0.581646   

                                                                            4     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       0.732576   
                                                   1.0        4.0       0.626332   
                                                   2.0        5.0       0.988916   
                                                   3.0        6.0       1.222748   
                                                   4.0        7.0       0.846210   

                                                                            5     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       0.163924   
                                                   1.0        4.0       0.208709   
                                                   2.0        5.0       0.125251   
                                                   3.0        6.0       0.067460   
                                                   4.0        7.0       0.000000   

                                                                            6     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       0.000000   
                                                   1.0        4.0       0.000000   
                                                   2.0        5.0       0.918078   
                                                   3.0        6.0       0.858694   
                                      

(304, 1024)

In [47]:
display(preds.head())
display(preds.shape)

0     \
file                                               start_time end_time              
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0      -11.252766   
                                                   1.0        4.0      -12.044468   
                                                   2.0        5.0      -11.033460   
                                                   3.0        6.0      -11.465073   
                                                   4.0        7.0      -12.021678   

                                                                             1     \
file                                               start_time end_time              
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0      -12.189754   
                                                   1.0        4.0      -12.687743   
                                                   2.0        5.0      -13.358752   
                                                   3.0        6.0      -14.568180   
                                                   4.0        7.0      -14.760959   

                                                                             2     \
file                                               start_time end_time              
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       -9.713304   
                                                   1.0        4.0       -9.642683   
                                                   2.0        5.0      -10.404150   
                                                   3.0        6.0      -10.562063   
                                                   4.0        7.0       -9.646517   

                                                                            3     \
file                                               start_time end_time             
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0      -7.946768   
                                                   1.0        4.0      -8.687905   
                                                   2.0        5.0      -6.832298   
                                                   3.0        6.0      -7.575617   
                                                   4.0        7.0      -7.553047   

                                                                             4     \
file                                               start_time end_time              
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       -9.549440   
                                                   1.0        4.0       -9.072952   
                                                   2.0        5.0      -12.357177   
                                                   3.0        6.0      -12.004049   
                                                   4.0        7.0      -11.713917   

                                                                             5     \
file                                               start_time end_time              
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0      -10.991865   
                                                   1.0        4.0      -12.235176   
                                                   2.0        5.0      -11.270983   
                                                   3.0        6.0      -12.029678   
                                                   4.0        7.0      -12.048661   

                                                                             6     \
file                                               start_time end_time              
/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2... 0.0        3.0       -9.110684   
                                                   1.0        4.0       -8.936573   
                                                   2.0        5.0      -10.952238   
                                                   3.0        6.0      -11.535090  

(304, 6522)

In [48]:
embed.reset_index(inplace=True)
embed.head()

,file,start_time,end_time,0,1,2,3,4,5,6,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef...,0.0,3.0,0.000000,0.174803,0.230387,0.175247,0.732576,0.163924,0.000000,...,0.009221,0.575789,0.092252,0.000000,0.134376,0.573795,0.158279,0.541457,0.000000,1.070122
1,/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef...,1.0,4.0,0.000000,0.106479,0.178000,0.270811,0.626332,0.208709,0.000000,...,0.000000,0.574536,0.085440,0.000000,0.079175,0.500204,0.036419,0.515172,0.024390,1.189686
2,/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef...,2.0,5.0,0.012751,0.317712,0.000000,0.604169,0.988916,0.125251,0.918078,...,0.000000,0.248399,0.014417,0.295829,0.575216,0.181418,0.206079,0.238235,0.152711,1.143211
3,/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef...,3.0,6.0,0.085640,0.482961,0.000000,0.635273,1.222748,0.067460,0.858694,...,0.000000,0.355565,0.009405,0.608990,0.649504,0.202187,0.231170,0.339462,0.176604,0.596274
4,/storage/home/hcoda1/9/mgustineli3/p-dsgt_clef...,4.0,7.0,0.000000,0.258623,0.000000,0.581646,0.846210,0.000000,0.404162,...,0.000000,0.507195,0.000000,0.245729,0.339466,0.384341,0.140074,0.196394,0.109323,0.860311


In [51]:
scratch_dir = "~/scratch/birdclef/data/2025"
embed_dir = (
    f"{scratch_dir}/subset-train_audio-infer-soundscape-cpu/BirdNET/parts/embed/"
)
embed_df = pd.read_parquet(embed_dir)
embed_df.head(10)

,file,start_time,end_time,0,1,2,3,4,5,6,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,0.0,3.0,0.000000,0.174803,0.230387,0.175247,0.732576,0.163924,0.000000,...,0.009221,0.575789,0.092252,0.000000,0.134376,0.573795,0.158279,0.541457,0.000000,1.070122
1,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,1.0,4.0,0.000000,0.106479,0.178000,0.270811,0.626332,0.208709,0.000000,...,0.000000,0.574536,0.085440,0.000000,0.079175,0.500204,0.036419,0.515172,0.024390,1.189686
2,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,2.0,5.0,0.012751,0.317712,0.000000,0.604169,0.988916,0.125251,0.918078,...,0.000000,0.248399,0.014417,0.295829,0.575216,0.181418,0.206079,0.238235,0.152711,1.143211
3,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,3.0,6.0,0.085640,0.482961,0.000000,0.635273,1.222748,0.067460,0.858694,...,0.000000,0.355565,0.009405,0.608990,0.649504,0.202187,0.231170,0.339462,0.176604,0.596274
4,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,4.0,7.0,0.000000,0.258623,0.000000,0.581646,0.846210,0.000000,0.404162,...,0.000000,0.507195,0.000000,0.245729,0.339466,0.384341,0.140074,0.196394,0.109323,0.860311
5,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,5.0,8.0,0.000000,0.082934,0.067922,0.409272,0.917499,0.148082,0.000000,...,0.005866,0.374507,0.015049,0.000000,0.179390,0.423340,0.112187,0.459117,0.112547,1.341081
6,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,6.0,9.0,0.338681,0.173996,0.146798,0.028051,1.731973,0.598605,0.160149,...,0.075030,0.405016,0.030888,0.123496,0.174462,0.717473,0.250377,0.638669,0.599883,0.820196
7,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,7.0,10.0,0.412865,0.503808,0.113986,0.412823,0.625809,0.456152,0.095818,...,0.167275,0.046151,0.055785,0.278683,0.073865,0.388267,0.361149,0.688742,0.367556,0.266666
8,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,8.0,11.0,0.133349,0.704304,0.135995,0.432688,0.208137,0.167350,0.069605,...,0.032013,0.000000,0.385816,0.000000,0.018100,0.400660,0.113311,0.698104,0.364200,0.284953
9,/storage/coda1/p-dsgt_clef2025/0/shared/birdcl...,9.0,12.0,0.232898,1.207564,0.038086,0.450695,0.177141,0.226787,0.159782,...,0.167688,0.000000,0.553128,0.162880,0.068582,0.306114,0.000000,0.565923,0.679935,0.117062


In [50]:
embed_df.shape

(192, 1027)